In [14]:
import pandas as pd
import json

In [15]:
#import csv into dataframe
path = f'NRI_Table_Counties.csv'
raw_df = pd.read_csv(path)
raw_df.head()

,OID_,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,POPULATION,...,WNTW_EALS,WNTW_EALR,WNTW_ALRB,WNTW_ALRP,WNTW_ALRA,WNTW_ALR_NPCTL,WNTW_RISKV,WNTW_RISKS,WNTW_RISKR,NRI_VER
0,1,C01001,Alabama,AL,1,Autauga,County,1,1001,58764,...,15.784587,Very Low,2.687716e-07,7.410082e-09,8.725777e-06,10.461158,8494.906508,12.217626,Very Low,March 2023
1,2,C01003,Alabama,AL,1,Baldwin,County,3,1003,231365,...,56.205509,Relatively Moderate,1.268231e-09,2.287120e-08,1.548360e-07,13.339523,65619.701638,52.083996,Relatively Low,March 2023
2,3,C01005,Alabama,AL,1,Barbour,County,5,1005,25160,...,18.632002,Relatively Low,5.788050e-07,2.347236e-08,7.606598e-07,16.125039,15501.730335,19.535476,Very Low,March 2023
3,4,C01007,Alabama,AL,1,Bibb,County,7,1007,22239,...,13.308573,Very Low,9.014679e-07,1.270300e-08,1.202015e-05,16.991643,7496.186940,11.104041,Very Low,March 2023
4,5,C01009,Alabama,AL,1,Blount,County,9,1009,58992,...,23.645930,Relatively Low,5.268425e-07,1.482016e-08,2.002965e-07,12.039616,17175.160729,21.444480,Very Low,March 2023


In [16]:
#select columns to keep
desired_columns = ['STATE', 'STATEABBRV', 'COUNTY', 'COUNTYTYPE', 'POPULATION', 'BUILDVALUE', 'AGRIVALUE', 'AREA', 'NRI_ID',
                'TRND_EVNTS', 'TRND_EXPB', 'TRND_EXP_AREA', 'TRND_EALB', 'TRND_EALA', 'TRND_EALS', 'TRND_EALR',
                'WFIR_EXPB', 'WFIR_EXP_AREA', 'WFIR_EALB', 'WFIR_EALA', 'WFIR_EALS', 'WFIR_EALR',
                'HRCN_EVNTS', 'HRCN_EXPB', 'HRCN_EXP_AREA', 'HRCN_EALB', 'HRCN_EALA', 'HRCN_EALS', 'HRCN_EALR',
                'HAIL_EVNTS', 'HAIL_EXPB', 'HAIL_EXP_AREA', 'HAIL_EALB', 'HAIL_EALA', 'HAIL_EALS', 'HAIL_EALR',
                 'ERQK_EXPB', 'ERQK_EXP_AREA', 'ERQK_EALB', 'ERQK_EALS', 'ERQK_EALR']
only_cols = raw_df.loc[:, desired_columns]

#replace NaN with 'none'
only_cols['COUNTYTYPE'] = only_cols['COUNTYTYPE'].replace({pd.NA: 'none', 'NaN': 'none'})

#drop all non-county rows minus louisiana - Louisiana consist of all 'parishes'
louisiana_df = only_cols[only_cols['STATE'].str.contains('Louisiana')]
norm_data_df = only_cols[only_cols['COUNTYTYPE'].str.contains('County')]

countytype_df = pd.concat([louisiana_df, norm_data_df], ignore_index=True)
countytype_df.reset_index(drop=True, inplace=True)
countytype_df.head()


,STATE,STATEABBRV,COUNTY,COUNTYTYPE,POPULATION,BUILDVALUE,AGRIVALUE,AREA,NRI_ID,TRND_EVNTS,...,HAIL_EXP_AREA,HAIL_EALB,HAIL_EALA,HAIL_EALS,HAIL_EALR,ERQK_EXPB,ERQK_EXP_AREA,ERQK_EALB,ERQK_EALS,ERQK_EALR
0,Louisiana,LA,Acadia,Parish,57475,1.082074e+10,115488675.0,664.173354,C22001,72.0,...,664.173354,183923.850649,43855.956993,70.564516,Relatively Low,1.082051e+10,664.173354,81822.1,45.403900,Very Low
1,Louisiana,LA,Allen,Parish,22708,3.900954e+09,22663141.0,773.856124,C22003,25.0,...,773.856124,1161.794314,10584.708894,20.998759,Very Low,3.900895e+09,773.856124,30465.9,31.847725,Very Low
2,Louisiana,LA,Ascension,Parish,126426,3.019167e+10,15019486.0,305.754813,C22005,19.0,...,305.754813,26836.091962,503.488900,53.442928,Relatively Low,3.019135e+10,305.754813,261690.0,64.221603,Relatively Low
3,Louisiana,LA,Assumption,Parish,21017,3.291188e+09,52318644.0,373.664791,C22007,17.0,...,373.664791,172.699911,1425.010989,14.950372,Very Low,3.291112e+09,373.664791,23783.1,29.464562,Very Low
4,Louisiana,LA,Avoyelles,Parish,39654,6.470116e+09,119631316.0,874.372686,C22009,47.0,...,874.372686,106917.773246,3591.049673,55.893300,Relatively Low,6.469990e+09,874.372686,92625.8,47.663262,Very Low


In [17]:
#drop County type col
clean_df = countytype_df.drop('COUNTYTYPE',axis=1)
clean_df.head()

,STATE,STATEABBRV,COUNTY,POPULATION,BUILDVALUE,AGRIVALUE,AREA,NRI_ID,TRND_EVNTS,TRND_EXPB,...,HAIL_EXP_AREA,HAIL_EALB,HAIL_EALA,HAIL_EALS,HAIL_EALR,ERQK_EXPB,ERQK_EXP_AREA,ERQK_EALB,ERQK_EALS,ERQK_EALR
0,Louisiana,LA,Acadia,57475,1.082074e+10,115488675.0,664.173354,C22001,72.0,1.082074e+10,...,664.173354,183923.850649,43855.956993,70.564516,Relatively Low,1.082051e+10,664.173354,81822.1,45.403900,Very Low
1,Louisiana,LA,Allen,22708,3.900954e+09,22663141.0,773.856124,C22003,25.0,3.900954e+09,...,773.856124,1161.794314,10584.708894,20.998759,Very Low,3.900895e+09,773.856124,30465.9,31.847725,Very Low
2,Louisiana,LA,Ascension,126426,3.019167e+10,15019486.0,305.754813,C22005,19.0,3.019167e+10,...,305.754813,26836.091962,503.488900,53.442928,Relatively Low,3.019135e+10,305.754813,261690.0,64.221603,Relatively Low
3,Louisiana,LA,Assumption,21017,3.291188e+09,52318644.0,373.664791,C22007,17.0,3.291188e+09,...,373.664791,172.699911,1425.010989,14.950372,Very Low,3.291112e+09,373.664791,23783.1,29.464562,Very Low
4,Louisiana,LA,Avoyelles,39654,6.470116e+09,119631316.0,874.372686,C22009,47.0,6.470116e+09,...,874.372686,106917.773246,3591.049673,55.893300,Relatively Low,6.469990e+09,874.372686,92625.8,47.663262,Very Low


In [18]:
#check for Nan Values
null_columns = clean_df.columns[clean_df.isnull().any()]
print(null_columns)

#replace null values with 0
clean_df.fillna(0,inplace=True)
clean_df.head()


Index(['HRCN_EVNTS', 'HRCN_EXPB', 'HRCN_EXP_AREA', 'HRCN_EALB', 'HRCN_EALA',
       'HRCN_EALS'],
      dtype='object')


,STATE,STATEABBRV,COUNTY,POPULATION,BUILDVALUE,AGRIVALUE,AREA,NRI_ID,TRND_EVNTS,TRND_EXPB,...,HAIL_EXP_AREA,HAIL_EALB,HAIL_EALA,HAIL_EALS,HAIL_EALR,ERQK_EXPB,ERQK_EXP_AREA,ERQK_EALB,ERQK_EALS,ERQK_EALR
0,Louisiana,LA,Acadia,57475,1.082074e+10,115488675.0,664.173354,C22001,72.0,1.082074e+10,...,664.173354,183923.850649,43855.956993,70.564516,Relatively Low,1.082051e+10,664.173354,81822.1,45.403900,Very Low
1,Louisiana,LA,Allen,22708,3.900954e+09,22663141.0,773.856124,C22003,25.0,3.900954e+09,...,773.856124,1161.794314,10584.708894,20.998759,Very Low,3.900895e+09,773.856124,30465.9,31.847725,Very Low
2,Louisiana,LA,Ascension,126426,3.019167e+10,15019486.0,305.754813,C22005,19.0,3.019167e+10,...,305.754813,26836.091962,503.488900,53.442928,Relatively Low,3.019135e+10,305.754813,261690.0,64.221603,Relatively Low
3,Louisiana,LA,Assumption,21017,3.291188e+09,52318644.0,373.664791,C22007,17.0,3.291188e+09,...,373.664791,172.699911,1425.010989,14.950372,Very Low,3.291112e+09,373.664791,23783.1,29.464562,Very Low
4,Louisiana,LA,Avoyelles,39654,6.470116e+09,119631316.0,874.372686,C22009,47.0,6.470116e+09,...,874.372686,106917.773246,3591.049673,55.893300,Relatively Low,6.469990e+09,874.372686,92625.8,47.663262,Very Low


In [19]:
#check for Nan Values
null_columns = clean_df.columns[clean_df.isnull().any()]
print(null_columns)

Index([], dtype='object')


In [20]:
#remove "C" from NRI_ID to get state and county code
clean_df['NRI_ID'] = clean_df['NRI_ID'].str.replace('C', '')

clean_df.head()

,STATE,STATEABBRV,COUNTY,POPULATION,BUILDVALUE,AGRIVALUE,AREA,NRI_ID,TRND_EVNTS,TRND_EXPB,...,HAIL_EXP_AREA,HAIL_EALB,HAIL_EALA,HAIL_EALS,HAIL_EALR,ERQK_EXPB,ERQK_EXP_AREA,ERQK_EALB,ERQK_EALS,ERQK_EALR
0,Louisiana,LA,Acadia,57475,1.082074e+10,115488675.0,664.173354,22001,72.0,1.082074e+10,...,664.173354,183923.850649,43855.956993,70.564516,Relatively Low,1.082051e+10,664.173354,81822.1,45.403900,Very Low
1,Louisiana,LA,Allen,22708,3.900954e+09,22663141.0,773.856124,22003,25.0,3.900954e+09,...,773.856124,1161.794314,10584.708894,20.998759,Very Low,3.900895e+09,773.856124,30465.9,31.847725,Very Low
2,Louisiana,LA,Ascension,126426,3.019167e+10,15019486.0,305.754813,22005,19.0,3.019167e+10,...,305.754813,26836.091962,503.488900,53.442928,Relatively Low,3.019135e+10,305.754813,261690.0,64.221603,Relatively Low
3,Louisiana,LA,Assumption,21017,3.291188e+09,52318644.0,373.664791,22007,17.0,3.291188e+09,...,373.664791,172.699911,1425.010989,14.950372,Very Low,3.291112e+09,373.664791,23783.1,29.464562,Very Low
4,Louisiana,LA,Avoyelles,39654,6.470116e+09,119631316.0,874.372686,22009,47.0,6.470116e+09,...,874.372686,106917.773246,3591.049673,55.893300,Relatively Low,6.469990e+09,874.372686,92625.8,47.663262,Very Low


In [21]:
#New column suffix names
new_suffix_names = {
    'EALB': 'expected_loss_building',
    'EXPB': 'exposed_building_value',
    'EALA': 'expected_loss_agri',
    'EALS': 'expected_loss_score',
    'EALR': 'expected_loss_rating',
    'EVNTS': 'EVNTS',
    'EXP_AREA': 'EXP_AREA'
}

# Columns that should be excluded from splitting (any column with underscore that doesn't have prefix)
excluded_columns = ['NRI_ID']

# Create a function to rename the columns based on the mapping, excluding 'NRI_ID' column
def rename_columns(column_name):
    #Exclude NRI_ID in split
    if column_name not in excluded_columns:
        if '_' in column_name:
            #return string index of last underscore for split
            last_underscore_index = column_name.rindex('_')
            #prefix = before last underscore
            prefix = column_name[:last_underscore_index]
            #suffix = after last underscore
            suffix = column_name[last_underscore_index + 1:]
            #column renamed as prefix_renamedSuffix
            if suffix in new_suffix_names:
                return f"{prefix}_{new_suffix_names[suffix]}"
        else:
            return column_name  #Not Prefix or suffix column remain unchanged
    return column_name

# Rename the columns 
clean_df.columns = [rename_columns(column) for column in clean_df.columns]
clean_df.columns

Index(['STATE', 'STATEABBRV', 'COUNTY', 'POPULATION', 'BUILDVALUE',
       'AGRIVALUE', 'AREA', 'NRI_ID', 'TRND_EVNTS',
       'TRND_exposed_building_value', 'TRND_EXP_AREA',
       'TRND_expected_loss_building', 'TRND_expected_loss_agri',
       'TRND_expected_loss_score', 'TRND_expected_loss_rating',
       'WFIR_exposed_building_value', 'WFIR_EXP_AREA',
       'WFIR_expected_loss_building', 'WFIR_expected_loss_agri',
       'WFIR_expected_loss_score', 'WFIR_expected_loss_rating', 'HRCN_EVNTS',
       'HRCN_exposed_building_value', 'HRCN_EXP_AREA',
       'HRCN_expected_loss_building', 'HRCN_expected_loss_agri',
       'HRCN_expected_loss_score', 'HRCN_expected_loss_rating', 'HAIL_EVNTS',
       'HAIL_exposed_building_value', 'HAIL_EXP_AREA',
       'HAIL_expected_loss_building', 'HAIL_expected_loss_agri',
       'HAIL_expected_loss_score', 'HAIL_expected_loss_rating',
       'ERQK_exposed_building_value', 'ERQK_EXP_AREA',
       'ERQK_expected_loss_building', 'ERQK_expected_loss_sco

In [22]:
#open geojson file and read data
with open("counties.geojson", "r") as file:
    geojson_data = json.load(file)


In [23]:
clean_df.columns

Index(['STATE', 'STATEABBRV', 'COUNTY', 'POPULATION', 'BUILDVALUE',
       'AGRIVALUE', 'AREA', 'NRI_ID', 'TRND_EVNTS',
       'TRND_exposed_building_value', 'TRND_EXP_AREA',
       'TRND_expected_loss_building', 'TRND_expected_loss_agri',
       'TRND_expected_loss_score', 'TRND_expected_loss_rating',
       'WFIR_exposed_building_value', 'WFIR_EXP_AREA',
       'WFIR_expected_loss_building', 'WFIR_expected_loss_agri',
       'WFIR_expected_loss_score', 'WFIR_expected_loss_rating', 'HRCN_EVNTS',
       'HRCN_exposed_building_value', 'HRCN_EXP_AREA',
       'HRCN_expected_loss_building', 'HRCN_expected_loss_agri',
       'HRCN_expected_loss_score', 'HRCN_expected_loss_rating', 'HAIL_EVNTS',
       'HAIL_exposed_building_value', 'HAIL_EXP_AREA',
       'HAIL_expected_loss_building', 'HAIL_expected_loss_agri',
       'HAIL_expected_loss_score', 'HAIL_expected_loss_rating',
       'ERQK_exposed_building_value', 'ERQK_EXP_AREA',
       'ERQK_expected_loss_building', 'ERQK_expected_loss_sco

In [24]:
# append dataframe data to geojson file
import json

# match geojson 'GEOID' to dataframe 'NRI_ID'
for feature in geojson_data['features']:
    geoid = feature['properties']['GEOID']
    nri_id_match = clean_df[clean_df['NRI_ID'] == geoid]
    if not nri_id_match.empty:
        # Convert the DataFrame row to a dictionary
        row_dict = nri_id_match.iloc[0].to_dict()

        # Remove 'NRI_ID' from the dictionary as we already have it in 'GEOID'
        del row_dict['NRI_ID']

        # Update each column as separate properties in the GeoJSON properties
        feature['properties'].update(row_dict)

# Save the updated JSON file - place this in dir with app.py
with open('updated_geojson_data3.json', 'w') as updated_json_file:
    json.dump(geojson_data, updated_json_file)


In [25]:
#create database for table import 
import psycopg2
import json
#change database creds to postgres db
with open('create_database_creds.json') as O:
    config = json.load(O)
username = config['user']
password = config['password']
db_host = config['host']
db_port = config['port']
db_ = 'weather_hazards_db'
conn = psycopg2.connect(
    host=db_host,
    port=db_port,
    user=username,
    password=password,
)
conn.autocommit = True
cursor = conn.cursor()
#create database to house table
cursor.execute(f"CREATE DATABASE {db_}")
cursor.close()
conn.close()

In [26]:
#export into csv if needed
clean_df.to_csv('Hazard_Data.csv')

In [28]:

from sqlalchemy import create_engine
#upload dataframe into postgres
# Replace these values with your PostgreSQL database credentials
db_username = config['user']
db_password = config['password']
db_host = config['host']
db_port = config['port']
db_name = 'weather_hazards_db'

#open engine
connection_path = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_path)

table_name = 'hazard_data'
#import data into db
clean_df.to_sql(table_name, engine, if_exists='replace', index=False)


71

In [ ]:
# create csv for pie_chart
pie_chart = clean_df[['STATE', 'TRND_EVNTS', 'HRCN_EVNTS', 'HAIL_EVNTS']]
pie_chart = pie_chart.groupby(['STATE']).mean()

pie_chart.head()

,TRND_EVNTS,HRCN_EVNTS,HAIL_EVNTS
STATE,,,
Alabama,37.537313,15.044776,113.432836
Arizona,10.933333,0.866667,8.200000
Arkansas,27.746667,3.413333,166.866667
California,6.189655,0.068966,3.603448
Colorado,27.437500,0.000000,117.906250


In [ ]:
#reset index
pie_chart = pie_chart.reset_index()

In [ ]:
#testing to_dict()
pie_chart [pie_chart['STATE'] == "Alabama"].to_dict()

{'STATE': {0: 'Alabama'},
 'TRND_EVNTS': {0: 37.53731343283582},
 'HRCN_EVNTS': {0: 15.044776119402986},
 'HAIL_EVNTS': {0: 113.43283582089552}}

In [ ]:
#export pie chart csv
pie_chart.to_csv("pie_chart.csv")